In [1]:
import numpy as np  # Load required libs
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
from tqdm.notebook import tqdm
from torch.nn import ReLU, Conv2d, BatchNorm2d, Sequential, AdaptiveAvgPool2d, Linear, MaxPool2d
from torch.nn.functional import one_hot
from typing import Tuple, List, Type, Dict, Any
from threading import Thread, Lock
from queue import Empty, Queue
from sklearn.utils import shuffle
from torch.autograd import Variable
import cv2
import os
import pickle
import datetime
import zipfile
import copy
import time
import sys
try:
    import adabelief_pytorch
except:
    !pip install adabelief_pytorch==0.2.0
    time.sleep(1)
    import adabelief_pytorch
import gc  # sometimes it is required to clean GPU memory in that way

In [2]:
import sys
sys.path.append('drive/MyDrive/GitHub/NN studying/NN_studying/metric learning - face recognition(like FaceID)/libs')

from dataset import Dataset, Cifar_dataset
from threading_data import Thread_data_processing
from train_val import train_model
from network_collection import Classificator, Combined_loss, CosFace

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # We'd like to use GPU
device

device(type='cuda', index=0)

In [4]:
train_gen = Cifar_dataset(batch_size=64, 
                          path_to_batches='drive/MyDrive/GitHub/NN studying/NN_studying/metric learning - face recognition(like FaceID)/data/cifar10',
                          train=True)

val_gen = Cifar_dataset(batch_size=64,
                        path_to_batches='drive/MyDrive/GitHub/NN studying/NN_studying/metric learning - face recognition(like FaceID)/data/cifar10',
                        train=False)

In [5]:
len(train_gen)

50000

In [6]:
len(val_gen)

10000

In [7]:
train_transforms = torchvision.transforms.Compose([torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                                   torchvision.transforms.RandomRotation(degrees=15)])
val_transforms = torchvision.transforms.Compose([torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), ])

In [8]:
model = Classificator(10, embedding_size=96, return_embedding=False).to(device)
print('Total number of trainable parameters : {}'.format(sum(parameter.numel() for parameter in model.parameters() if parameter.requires_grad)))

Total number of trainable parameters : 363552


In [14]:
for parameter in model.parameters():
    parameter.requires_grad_(True)
print('Total number of trainable parameters : {}'.format(sum(parameter.numel() for parameter in model.parameters() if parameter.requires_grad)))

Total number of trainable parameters : 43278432


In [11]:
datagiver = Thread_data_processing(train_generator = train_gen, 
                                   val_generator = val_gen,
                                   device = device, 
                                   img_transform_train = train_transforms, 
                                   img_transform_val = val_transforms,
                                   ram_queue_length=24,
                                   cuda_queue_length=10,
                                   num_workers_ram=3,
                                   num_workers_cuda=1)
datagiver.start()

process has been started
process has been started
process has been started
process has been started


In [15]:
adabelief_params = {'lr':0.001,
'betas':(0.9, 0.999),
'eps':1e-8,
'weight_decouple':True,
'rectify':False,
'weight_decay':1e-2,
'fixed_decay':False,
'amsgrad':False}

train_model(model = model, 
            datagiver = datagiver,
            path_to_save = 'drive/MyDrive/GitHub/NN studying/NN_studying/metric learning - face recognition(like FaceID)/best_models/',
            loss_function = CosFace(scale=32, margin=0.35),
            optimizer_params = adabelief_params,
            max_epochs = 20,
            early_stopping_patience = 3,
            loss_progression = None)

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

threads has been frozen
threads has been frozen
threads has been frozen
thread has been frozen
Number of hot threads - 0
process has been started
process has been startedprocess has been startedprocess has been started


4 processes have been starter


  0%|          | 0/782 [00:00<?, ?it/s]

...

threads has been frozen
threads has been frozen
threads has been frozen
thread has been frozen
Number of hot threads - 0
process has been started
process has been started
process has been started
process has been started
4 processes have been starter
correct - 0.8420581210191083 , loss - 0.46901773988820944
Best model yet, saving
threads has been frozen
threads has been frozen
threads has been frozen
thread has been frozen
Number of hot threads - 0
process has been startedprocess has been started
process has been started

3 processes have been starter
process has been started

  0%|          | 0/782 [00:00<?, ?it/s]


threads has been frozen
threads has been frozen
threads has been frozen
thread has been frozen
Number of hot threads - 0
process has been started
process has been startedprocess has been started

process has been started
4 processes have been starter
correct - 0.8559912420382165 , loss - 0.44741348343290344
Best model yet, saving
Number of hot threads - 4
threads has been frozen
threads has been frozen
threads has been frozen
thread has been frozen
Number of hot threads - 0
process has been startedprocess has been started

process has been started
process has been started
4 processes have been starter


  0%|          | 0/782 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [4]:
DATA_PATH = ''
if os.path.exists(DATA_PATH +'/CASIA-WebFace_crop') or os.path.exists(DATA_PATH +'CASIA-WebFace_crop'):
    pass
else:
    data_zip = zipfile.ZipFile('drive/MyDrive/GitHub/NN studying/NN_studying/metric learning - face recognition(like FaceID)/data/archive.zip')
    data_zip.extractall(DATA_PATH)
    data_zip.close()

In [5]:
train_gen, val_gen = Dataset.make_train_val_split(batch_size=16,
                                                  ratio=0.8,
                                                  path='',
                                                  least_size = 50)

print('Num classes train - {}'.format(len(train_gen.ID_to_number)))
print('Num classes val - {}'.format(len(val_gen.ID_to_number)))
print('Train data len - {}'.format(len(train_gen.pathes_list)))

Num classes train - 596
Num classes val - 596
Train data len - 66315
